In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-11-20 18:33:31--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  66.8MB/s    in 0.4s    

2024-11-20 18:33:31 (66.8 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'},
)

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'int8'},
)

In [4]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271379 non-null  object
 1   title   271379 non-null  object
 2   author  271377 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [5]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   user    1149780 non-null  int32 
 1   isbn    1149780 non-null  object
 2   rating  1149780 non-null  int8  
dtypes: int32(1), int8(1), object(1)
memory usage: 14.3+ MB


In [6]:
df = df_books.merge(df_ratings, on="isbn", how="right")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   isbn    1149780 non-null  object
 1   title   1031175 non-null  object
 2   author  1031173 non-null  object
 3   user    1149780 non-null  int32 
 4   rating  1149780 non-null  int8  
dtypes: int32(1), int8(1), object(3)
memory usage: 31.8+ MB


In [8]:
user_unique = df["user"].value_counts()
user_unique = user_unique[user_unique >= 200].index
isbn_unique = df["isbn"].value_counts()
isbn_unique = isbn_unique[isbn_unique >= 100].index

In [9]:
df_f = df.loc[(df["user"].isin(user_unique.values)) & (df["isbn"].isin(isbn_unique.values))]
df_pivot = (
    df_f.drop_duplicates(["title", "user"])
        .pivot(index='title', columns='user', values='rating')
        .fillna(0)
)
matrix = csr_matrix(df_pivot.values)

In [12]:
# function to return recommended books - this will be tested
def get_recommends(book: str, num_of_preds: int = 5):
    model = NearestNeighbors(n_neighbors=num_of_preds, metric="cosine")
    model.fit(matrix)
    book_probs, book_index = model.kneighbors([df_pivot.loc[book]])
    recom_str = df_pivot.iloc[np.flip(book_index[0])].index.to_list()
    recom_dist = list(np.flip(book_probs[0]))
    recommended_books = list(map(list, zip(recom_str, recom_dist)))
    return [book, recommended_books]

In [13]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.7708583572697412], ['The Surgeon', 0.7699410973804288], ['I Know This Much Is True', 0.7677075092617776], ['The Lovely Bones: A Novel', 0.7234864549790632], ["Where the Heart Is (Oprah's Book Club (Paperback))", 7.771561172376096e-16]]]
You passed the challenge! 🎉🎉🎉🎉🎉
